In [1]:
%matplotlib inline
from sympy import symbols, Eq, solve
import math
from itertools import combinations_with_replacement , permutations, combinations, count, chain
import numpy as np
import time
import functools
from tqdm.notebook import trange, tqdm
import ipywidgets as widgets
import matplotlib.pyplot as plt
from collections import Counter, deque
import more_itertools as mit
import string
import ast
import networkx as nx
import pyvista
import plotly.graph_objects as go
from IPython.display import display
import operator
import re
from typing import TypeVar, Iterable, Tuple

In [2]:
sample_encoded = ['1=-0-2', '12111', '2=0=', '21', '2=01', '111', '20012', '112', '1=-1=', '1-12', '12', '1=', '122']
sample_decoded = [1747, 906, 198, 11, 201, 31, 1257, 32, 353, 107, 7, 3, 37]

In [26]:
# problem inpout. Part 1 = 2=01-0-2-0=-0==-1=01
problem_input = ['121=0=-', '1==02020022=', '1==2==20--2==', '1-11-', '1==02=2=02=-1-00-=0', '1011=-=--00', '22-1-1=0--0=', '1-=121-00=', '1=1=-=10====10', '1=-=-21002-2=--', '211-2=2=-', '1=-1=10', '11221', '2-21-20', '1-0-', '12221=0-0-1=', '101-022121111-', '11=01-==0211', '1-2-00-=--11', '1=-12=-=-2=1==02', '1=-0111===202', '20011', '2=22', '1=2=12-===-=', '202=-2201=-2=', '2=-0221', '10=20-=2=--', '1=01202=', '202', '12211=222=', '2=1=1-0211-=0=', '1-10', '112=211112', '11==-100-', '1=21220=-122--2-2-', '11-220', '122=1110220==1', '1=1-2210-21-=120102', '1=0202=1=', '11=2100==2-21121', '2-21-=', '2=00210-', '1==--2', '1-', '1==', '1220-1=22-=', '1=1-==21021-=10', '2=02=02000==1-', '1--210-0-0', '12-200==1=0-2=-0-', '12110-020-2-1--', '1-2', '20=2=--12=120', '112=20201', '1=021-0-=2---', '21202--1=-1=2--', '101=', '22=0---=-', '21', '10-1020', '12=2-==-0-=12=000=', '1-212-', '200=1--0120=2==201', '200-', '2=0', '1-1=1110211', '1---1==2', '1-2122==10--=0=10', '1=0-0-2-=01-=220', '2-0==00001002-=', '1-0--0001-2=0=0-', '1-02--01=-1', '1=101121=0110021-', '22-22===0-2', '1=-0-----202=212201', '120-00--10-=202=', '1=00==', '111022--0012=', '11-2-0=01-=-02', '1-011=-2', '2=021-=--1', '1--212000-2', '1=-000-', '1-=-==0', '20=0012110', '12=21-1-0', '1=0021=1010=0=2', '11-0-==01002', '1102-20=-', '102', '12122', '1-100121=', '12=-1==20=20=20222', '1-=-00--0--1-', '1===-22-2201=01-=-2', '1==0-11--==2=21=1-=1', '20=0=20-110-1201-=2', '112', '22=-', '2=2-2021', '11100=', '211-', '10000==10-000--0', '12', '1-1---1=111=', '11=-=102===-1-1=-', '20210-2-2-==11', '1012', '1-0=-121==-0', '1===0-0=-0=-=210', '12--1-2---021=-=', '1000', '2-2011-', '1--201001=002', '2--', '1---102=2=10110', '1=0=2=011-=', '12=--2-=1-0']

In [7]:
def decode(st):
    total = 0
    for i, c in enumerate(reversed(st)):
        if c.isnumeric():
            t = int(c)
        elif c == '=':
            t = -2
        else:
            t = -1
            
        total += (5**i)*t
    
    return total
        
        
    
print(decode('1=')) # 3
print('-----')
decode('2=-01')  # 976

3
-----


976

In [25]:
charmap = {0:'0', 1:'1', 2:'2', 3:'=', 4:'-'}
addmap = {0:0, 1:0, 2:0, 3:1, 4:1}

def encode(num):
    r = num % 5
    t = num // 5
    t += addmap[r]
    s = charmap[r]
    while t > 0:
        r = t % 5
        t = t // 5 
        t += addmap[r]
        s = charmap[r] + s
    return s
        
    
tnum = 13
e = encode(tnum)
d = decode(e)
print(tnum,':  encoded', e, '=' if tnum == d else '!=', 'decoded', d)


13 :  encoded 1== = decoded 13


In [30]:
# quick test function for the encoding/decoding
for i in range(0, len(sample_decoded)):
    if decode(sample_encoded[i]) != sample_decoded[i]:
        print("This one had trouble decoding!", decode(sample_encoded[i]), ' != ', sample_decoded[i])
    else:
        print(sample_decoded[i], ' \t= ', sample_encoded[i])
        
    if encode(sample_decoded[i]) != sample_encoded[i]:
        print("This one had trouble encoding!", encode(sample_decoded[i]), ' != ', sample_encoded[i])
    else:
        print(sample_decoded[i], ' \t= ', sample_encoded[i])

1747  	=  1=-0-2
1747  	=  1=-0-2
906  	=  12111
906  	=  12111
198  	=  2=0=
198  	=  2=0=
11  	=  21
11  	=  21
201  	=  2=01
201  	=  2=01
31  	=  111
31  	=  111
1257  	=  20012
1257  	=  20012
32  	=  112
32  	=  112
353  	=  1=-1=
353  	=  1=-1=
107  	=  1-12
107  	=  1-12
7  	=  12
7  	=  12
3  	=  1=
3  	=  1=
37  	=  122
37  	=  122


In [28]:
nums = problem_input
#nums = sample_encoded    # c0mment this out to get real answer
 
total = 0
for n in nums:
    total += decode(n)
print(encode(total))


2=01-0-2-0=-0==-1=01
